In [1]:
sc.version

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1621292081500_0003,spark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res1: String = 2.4.7-amzn-1


In [2]:
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator  
import org.apache.spark.mllib.evaluation.MulticlassMetrics  
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics  
import org.apache.spark.ml.classification.RandomForestClassifier  
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit, CrossValidator}  
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer, OneHotEncoderEstimator}  
import org.apache.spark.ml.linalg.Vectors  
import org.apache.spark.ml.Pipeline  
import org.apache.log4j._

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.mllib.evaluation.MulticlassMetrics
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
import org.apache.spark.ml.classification.RandomForestClassifier
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit, CrossValidator}
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer, OneHotEncoderEstimator}
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.Pipeline
import org.apache.log4j._


In [3]:
Logger.getLogger("org").setLevel(Level.ERROR) 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@2e1e4f16


# Load Data

Put the data into an S3 bucket and load it from there into a Spark DataFrame.

Grab the dataset from the Harvard Dataverse: https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/26147


In [5]:
var dataset="HXPC13_DI_v3_11-13-2019.csv"
var bucket="s3://sagemaker-london/"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dataset: String = HXPC13_DI_v3_11-13-2019.csv
bucket: String = s3://sagemaker-london/


In [6]:
val data = spark.read.option("header","true").
    option("inferSchema","true").
    format("csv").
    load(bucket + dataset)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

data: org.apache.spark.sql.DataFrame = [course_id: string, userid_DI: string ... 18 more fields]


In [7]:
val df = (data.select(data("certified").as("label"), 
          $"registered", $"viewed", $"explored", 
          $"final_cc_cname_DI", $"gender", $"nevents", 
          $"ndays_act", $"nplay_video", $"nchapters", $"nforum_posts"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df: org.apache.spark.sql.DataFrame = [label: int, registered: int ... 9 more fields]


In [8]:
val indexer1 = new StringIndexer().
    setInputCol("final_cc_cname_DI").
    setOutputCol("countryIndex").
    setHandleInvalid("keep") 
val indexed1 = indexer1.fit(df).transform(df)

val indexer2 = new StringIndexer().
    setInputCol("gender").
    setOutputCol("genderIndex").
    setHandleInvalid("keep")
val indexed2 = indexer2.fit(indexed1).transform(indexed1)

// one hot encoding
val encoder = new OneHotEncoderEstimator().
  setInputCols(Array("countryIndex", "genderIndex")).
  setOutputCols(Array("countryVec", "genderVec"))
val encoded = encoder.fit(indexed2).transform(indexed2)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

indexer1: org.apache.spark.ml.feature.StringIndexer = strIdx_78d7a9cadabf
indexed1: org.apache.spark.sql.DataFrame = [label: int, registered: int ... 10 more fields]
indexer2: org.apache.spark.ml.feature.StringIndexer = strIdx_982088909bd7
indexed2: org.apache.spark.sql.DataFrame = [label: int, registered: int ... 11 more fields]
encoder: org.apache.spark.ml.feature.OneHotEncoderEstimator = oneHotEncoder_7ed5d0900abb
encoded: org.apache.spark.sql.DataFrame = [label: int, registered: int ... 13 more fields]


In [9]:
val nanEvents = encoded.groupBy("nevents").count().orderBy($"count".desc)
for (line <- nanEvents){
    println(line)
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

nanEvents: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [nevents: int, count: bigint]


In [10]:
val neventsMedianArray = encoded.stat.approxQuantile("nevents", Array(0.5), 0)
val neventsMedian = neventsMedianArray(0)

val ndays_actMedianArray = encoded.stat.approxQuantile("ndays_act", Array(0.5), 0)
val ndays_actMedian = ndays_actMedianArray(0)

val nplay_videoMedianArray = encoded.stat.approxQuantile("nplay_video", Array(0.5), 0)
val nplay_videoMedian = nplay_videoMedianArray(0)

val nchaptersMedianArray = encoded.stat.approxQuantile("nchapters", Array(0.5), 0)
val nchaptersMedian = nchaptersMedianArray(0)

// replace 
val filled = encoded.na.fill(Map(
  "nevents" -> neventsMedian, 
  "ndays_act" -> ndays_actMedian, 
  "nplay_video" -> nplay_videoMedian, 
  "nchapters" -> nchaptersMedian))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

neventsMedianArray: Array[Double] = Array(17.0)
neventsMedian: Double = 17.0
ndays_actMedianArray: Array[Double] = Array(2.0)
ndays_actMedian: Double = 2.0
nplay_videoMedianArray: Array[Double] = Array(17.0)
nplay_videoMedian: Double = 17.0
nchaptersMedianArray: Array[Double] = Array(2.0)
nchaptersMedian: Double = 2.0
filled: org.apache.spark.sql.DataFrame = [label: int, registered: int ... 13 more fields]


In [12]:
// Set the input columns as the features we want to use
val assembler = (new VectorAssembler().setInputCols(Array(
  "viewed", "explored", "nevents", "ndays_act", "nplay_video", 
  "nchapters", "nforum_posts", "countryVec", "genderVec")).
   setOutputCol("features"))

// Transform the DataFrame
val output = assembler.transform(filled).select($"label",$"features")
                 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_9a3c7dbdc7db
output: org.apache.spark.sql.DataFrame = [label: int, features: vector]


In [13]:
// Splitting the data by create an array of the training and test data
val Array(training, test) = output.select("label","features").
                            randomSplit(Array(0.7, 0.3), seed = 12345)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

training: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: int, features: vector]
test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: int, features: vector]


In [14]:
// create the model
val rf = new RandomForestClassifier()

// create the param grid
val paramGrid = new ParamGridBuilder().
  addGrid(rf.numTrees,Array(20,50,100)).
  build()

// create cross val object, define scoring metric
val cv = new CrossValidator().
  setEstimator(rf).
  setEvaluator(new MulticlassClassificationEvaluator().setMetricName("weightedRecall")).
  setEstimatorParamMaps(paramGrid).
  setNumFolds(3).
  setParallelism(2)

// You can then treat this object as the model and use fit on it.
val model = cv.fit(training)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rf: org.apache.spark.ml.classification.RandomForestClassifier = rfc_90b9f8aa8812
paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	rfc_90b9f8aa8812-numTrees: 20
}, {
	rfc_90b9f8aa8812-numTrees: 50
}, {
	rfc_90b9f8aa8812-numTrees: 100
})
cv: org.apache.spark.ml.tuning.CrossValidator = cv_fd6bca14c254
model: org.apache.spark.ml.tuning.CrossValidatorModel = cv_fd6bca14c254


In [16]:
val results = model.transform(test).select("features", "label", "prediction")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

results: org.apache.spark.sql.DataFrame = [features: vector, label: int ... 1 more field]


In [17]:
val predictionAndLabels = results.
    select($"prediction",$"label").
    as[(Double, Double)].
    rdd

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

predictionAndLabels: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[694] at rdd at <console>:40


In [18]:
// Instantiate a new metrics objects
val bMetrics = new BinaryClassificationMetrics(predictionAndLabels)
val mMetrics = new MulticlassMetrics(predictionAndLabels)
val labels = mMetrics.labels

// Print out the Confusion matrix
println("Confusion matrix:")
println(mMetrics.confusionMatrix)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bMetrics: org.apache.spark.mllib.evaluation.BinaryClassificationMetrics = org.apache.spark.mllib.evaluation.BinaryClassificationMetrics@108a3e94
mMetrics: org.apache.spark.mllib.evaluation.MulticlassMetrics = org.apache.spark.mllib.evaluation.MulticlassMetrics@6922ea64
labels: Array[Double] = Array(0.0, 1.0)
Confusion matrix:
99695.0  231.0  
1215.0   709.0  


In [19]:
// Precision by label
labels.foreach { l =>
  println(s"Precision($l) = " + mMetrics.precision(l))
}

// Recall by label
labels.foreach { l =>
  println(s"Recall($l) = " + mMetrics.recall(l))
}

// False positive rate by label
labels.foreach { l =>
  println(s"FPR($l) = " + mMetrics.falsePositiveRate(l))
}

// F-measure by label
labels.foreach { l =>
  println(s"F1-Score($l) = " + mMetrics.fMeasure(l))
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Precision(0.0) = 0.9879595679318204
Precision(1.0) = 0.7542553191489362
Recall(0.0) = 0.9976882893341072
Recall(1.0) = 0.3685031185031185
FPR(0.0) = 0.6314968814968815
FPR(1.0) = 0.0023117106658927604
F1-Score(0.0) = 0.9928000956003903
F1-Score(1.0) = 0.4951117318435754


In [20]:
// Precision by threshold
val precision = bMetrics.precisionByThreshold
precision.foreach { case (t, p) =>
  println(s"Threshold: $t, Precision: $p")
}

// Recall by threshold
val recall = bMetrics.recallByThreshold
recall.foreach { case (t, r) =>
  println(s"Threshold: $t, Recall: $r")
}

// Precision-Recall Curve
val PRC = bMetrics.pr

// F-measure
val f1Score = bMetrics.fMeasureByThreshold
f1Score.foreach { case (t, f) =>
  println(s"Threshold: $t, F-score: $f, Beta = 1")
}

val beta = 0.5
val fScore = bMetrics.fMeasureByThreshold(beta)
f1Score.foreach { case (t, f) =>
  println(s"Threshold: $t, F-score: $f, Beta = 0.5")
}

// AUPRC
val auPRC = bMetrics.areaUnderPR
println("Area under precision-recall curve = " + auPRC)

// Compute thresholds used in ROC and PR curves
val thresholds = precision.map(_._1)

// ROC Curve
val roc = bMetrics.roc

// AUROC
val auROC = bMetrics.areaUnderROC
println("Area under ROC = " + auROC)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

precision: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[713] at map at BinaryClassificationMetrics.scala:217
recall: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[714] at map at BinaryClassificationMetrics.scala:217
PRC: org.apache.spark.rdd.RDD[(Double, Double)] = UnionRDD[717] at union at BinaryClassificationMetrics.scala:110
f1Score: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[718] at map at BinaryClassificationMetrics.scala:217
beta: Double = 0.5
fScore: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[719] at map at BinaryClassificationMetrics.scala:217
auPRC: Double = 0.5220650321042403
Area under precision-recall curve = 0.5220650321042403
thresholds: org.apache.spark.rdd.RDD[Double] = MapPartitionsRDD[725] at map at <console>:39
roc: org.apache.spark.rdd.RDD[(Double, Double)] = UnionRDD[729] at UnionRDD at BinaryClassificationMetrics.scala:90
auROC: Double = 0.6830957039186129
Area under ROC = 0.6830957039186129
